In [ ]:
import os
import sys
sys.path.append(os.path.dirname(os.path.realpath('__file__')) + '/../../../')
from acaisdk.file import File
from acaisdk.project import Project
from acaisdk.fileset import FileSet
from acaisdk.job import Job
from acaisdk.meta import *
from acaisdk.utils import utils
from acaisdk import credentials

utils.DEBUG = True  # print debug messages
workspace = os.path.dirname(os.path.realpath('__file__'))  # get current directory

In [ ]:
# Make your changes here
project_id = "your_project_name"
root_token = 'Ask sys guys for the root token'
project_admin = 'allen'
user = 'albert'

In [ ]:
# Create project and user
r = Project.create_project(project_id, root_token, project_admin)
# Login is done automatically upon user creation
r = Project.create_user(project_id, r['project_admin_token'], user)  

In [ ]:
# you can inspect your token in multiple ways
print(r['user_token'])
print(os.environ['ACAI_TOKEN'])
print(credentials.get_credentials())

In [ ]:
# Upload code
code = os.path.join(workspace, 'wordcount.zip')
File.upload({code: '/wordcount.zip'})

In [ ]:
credentials.login('8gyszGh9LaR1uk90cfrdXpahZZxXFSFt')

In [ ]:
# Upload input files and create file set
input_dir = os.path.join(workspace, 'Shakespeare')
File.convert_to_file_mapping([input_dir], 'Shakespeare/') \
    .files_to_upload \
    .upload() \
    .as_new_file_set('shakespeare.works')

In [ ]:
# Run a job
job_setting = {
    "v_cpu": "0.2",
    "memory": "64Mi",
    "gpu": "0",
    "command": "mkdir -p ./my_output/ && (cat Shakespeare/* | python3 wordcount.py ./my_output/)",
    "container_image": "pytorch/pytorch",
    'input_file_set': 'shakespeare.works',
    'output_path': './my_output/',
    'code': '/wordcount.zip',
    'description': 'count some words from Shakespeare works',
    'name': 'my_acai_job'
}

j = Job().with_attributes(job_setting).run()

In [ ]:
# Take a look at what's in the output folder
File.list_dir('/my_output')

In [ ]:
# Download the result to local device
File.download({'/my_output/wordcount.txt': workspace})

In [ ]:
# Inspect the 50 most frequent words
with open(os.path.join(workspace, 'wordcount.txt')) as f:
    for i, l in enumerate(f):
        if i >= 50:
            break
        print(l, end='')